# Recommendation Systems

A recommendation system predicts how much a user will like an item they haven't interacted with yet.

Examples:

- Netflix recommends movies

- Spotify recommends songs

Formally,

$f(\text{user, item}) = \text{score}$

Score here can be rating, click probability etc.

#### Types of recommendation systems

**Content Based**

- Recommend items like other items you have liked (genre, director, description text etc.)

**Collaborative Filtering**

- Recommend based on other users' behavior. So recommend items that users like you liked.

- Use **embeddings** for both the item and user

- Dot product theses embeddings per user and item to get the prediction

**Modern methods**

- Combine the 2 methods above

- You can also add additional context (device, time, location)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


#### Example: Matrix Factorization

| User / Item | Movie A | Movie B | Movie C |
| ----------- | ------- | ------- | ------- |
| User 1      | 5       | -       | 1       |
| User 2      | -       | 4       | 2       |
| User 3      | 1       | 5       | -       |

We have a sparse matrix. We learn latent embeddings for both item and user.

In [ ]:
# userid, itemid, rating
data = torch.tensor([
    [0, 0, 5.0],
    [0, 2, 1.0],
    [1, 1, 4.0],
    [1, 2, 2.0],
    [2, 0, 1.0],
    [2, 1, 5.0],
])

class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, embedding_dim)
        self.item_emb = nn.Embedding(num_items, embedding_dim)

    def forward(self, user_ids, item_ids):
        u = self.user_emb(user_ids)
        i = self.item_emb(item_ids)
        return (u * i).sum(dim=1) # row-wise dot product


In [ ]:
num_users = 3
num_items = 3
model = MatrixFactorization(num_users, num_items, embedding_dim=2)

optimizer = optim.Adam(model.parameters(), lr=0.05)
loss_fn = nn.MSELoss()
users = data[:, 0].long()
items = data[:, 1].long()
ratings = data[:, 2]

# training loop
for epoch in range(201):
    optimizer.zero_grad()
    preds = model(users, items)
    loss = loss_fn(preds, ratings)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss {loss.item():.4f}")


Epoch 0, Loss 17.3294
Epoch 100, Loss 0.1567
Epoch 200, Loss 0.0000


#### Printing User 1 predicted scores:

In [31]:
user_id = torch.tensor([0])
all_items = torch.arange(num_items)

scores = model(user_id.repeat(num_items), all_items)
print(scores)

tensor([ 5.0007, 11.4591,  0.9999], grad_fn=<SumBackward1>)


Now we can recommend items with the highest scores

Dot products cannot capture non-linear interactions $\rightarrow$ MLP

#### Example: Neural Collaborative Filtering

In [ ]:
class NeuralCF(nn.Module):
    def __init__(self, num_users, num_items, emb_dim):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)

        self.mlp = nn.Sequential(
            nn.Linear(2 * emb_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, user_ids, item_ids):
        u = self.user_emb(user_ids)
        i = self.item_emb(item_ids)
        x = torch.cat([u, i], dim=1)
        x = self.mlp(x).squeeze()
        return x


While this captures nonlinearities, the MLP in the forward pass adds a lot of latency while serving recommendations if there are many items.

#### Example: Two-Tower Models:

Two-tower models use neural networks to learn user and item embeddings independently. The final relevance score is computed using a dot product.

Because item embeddings do not depend on the user, they can be precomputed and stored in a vector index. At inference time, the system computes a single user embedding and retrieves the most similar items using approximate nearest-neighbor (ANN) search, reducing latency compared to scoring every item individually

In [33]:
class TwoTowerModel(nn.Module):
    def __init__(self, num_users, num_items, emb_dim):
        super().__init__()

        # User tower
        self.user_tower = nn.Sequential(
            nn.Embedding(num_users, emb_dim),
            nn.Linear(emb_dim, emb_dim),
            nn.ReLU()
        )

        # Item tower
        self.item_tower = nn.Sequential(
            nn.Embedding(num_items, emb_dim),
            nn.Linear(emb_dim, emb_dim),
            nn.ReLU()
        )

    def forward(self, user_ids, item_ids):
        u_vec = self.user_tower(user_ids)
        i_vec = self.item_tower(item_ids)

        return (u_vec * i_vec).sum(dim=1)